In [1]:
import cv2
import os
import time
import uuid
import handTrackingModule as htm
import numpy as np


In [2]:
IMAGE_PATH = 'train_data'
labels = ['ok01']
number_imgs = 20
contours_size = 100
padding = 55
detector = htm.handDetector(detectionCon=0.75)


In [3]:

def reshape_contours(x0, x1, y0, y1, padding):
    if(abs(x0 - x1) > abs(y0 - y1)):
        padding_0 = padding
        x0_new = x0 - padding_0
        x1_new = x1 + padding_0
        len_axis_x = abs(x0 - x1) + 2*padding_0
        padding_1 = (len_axis_x - abs(y0 - y1))//2
        y0_new, y1_new = y0 - padding_1, y1 + padding_1

        return x0_new, x1_new, y0_new, y1_new
    else:
        padding_0 = padding
        y0_new = y0 - padding_0
        y1_new = y1 + padding_0
        len_axis_y = abs(y0 - y1) + 2*padding_0
        padding_1 = (len_axis_y - abs(y0 - y1))//2
        x0_new, x1_new = x0 - padding_1, x1 + padding_1

        return x0_new, x1_new, y0_new, y1_new

In [4]:
for label in labels:
    os.mkdir(IMAGE_PATH + '\\' +  label)
    cap = cv2.VideoCapture(0)
    print('collecting label: {}'.format(label))
    time.sleep(5)
    for imgNum in range(number_imgs):
        ret, frame = cap.read()
        _, landmarks = detector.findHands(frame)
        # find hand area
        try:
            min_x = 0
            min_y = 0
            max_x = 0
            max_y = 0
            if len(landmarks) > 0:
                xx, yy = zip(*landmarks)
                min_x = min(xx)
                min_y = min(yy)
                max_x = max(xx)
                max_y = max(yy)
                # cv2.rectangle(image, (min_x, min_y),
                #             (max_x, max_y), (0, 255, 255), 2)
                if max_x//min_x == max_y//min_y:
                    cv2.rectangle(frame, (min_x-padding, min_y - padding),
                                (max_x + padding, max_y + padding), (0, 255, 0), 2)
                    min_x, max_x, min_y, max_y =  min_x-padding, max_x + padding, min_y - padding, max_y + padding

                else:
                    min_x, max_x, min_y, max_y = reshape_contours(
                        min_x, max_x, min_y, max_y, padding)
                    cv2.rectangle(frame, (min_x, min_y),
                                (max_x, max_y), (0, 255, 0), 2)
        except Exception as e:
            print(e)
            # PROCESS IMAGE
        try:
            cropImage = frame[min_y:max_y, min_x:max_x]
            cropImage = np.array(cropImage)
            # cropImage = cv2.resize(cropImage, (256, 256))
            hsv = cv2.cvtColor(cropImage, cv2.COLOR_BGR2HSV)
            mask = cv2.inRange(hsv, (58,255,255), (62, 255, 255))
            cv2.imshow("Mask", mask)
            #write frame in to folder label
            imageName = os.path.join(IMAGE_PATH, label, label+'.'+'{}.jpg'.format(str(uuid.uuid1())))
            cv2.imwrite(imageName, mask)
            cv2.imshow('frame', frame)
        except Exception as e: 
            print(e)
            pass
        time.sleep(2)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()

collecting label: ok01
